In [1]:
import os

import sklearn
import pandas as pd
import numpy as np
import cv2
import pickle

In [2]:
from models.inception import build_inceptionv3_based_classifier


Using TensorFlow backend.


In [3]:
classes_file_name = './data/classes-trainable.csv'
classes_file = pd.read_csv(classes_file_name)

In [4]:
all_labels = classes_file['label_code']

In [5]:
# set the number of labels which will be used as an output layer size for a model
num_labels = len(all_labels)

# build the index dictionary based on the labels collection
labels_index = {label:idx for idx, label in enumerate(all_labels)}



In [6]:
test_images_dir = './data/stage_1_test_images/'
test_images = os.listdir(test_images_dir)

print (test_images[0])

6b704b475a2b4245336c4d3d.jpg


In [8]:
input_shape = (450, 450, 3)

In [9]:
model = build_inceptionv3_based_classifier(input_shape, num_labels)

87916544/87910968 [==============================] - 23s 0us/step


In [10]:
model.load_weights('inception_model.h5')

/Users/Viktor/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 12, 12, 2048)      21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 7178)              14707722  
Total params: 36,510,506
Trainable params: 36,476,074
Non-trainable params: 34,432
_________________________________________________________________


In [12]:
n_channels = 3

In [13]:
def normalize(img):
    return (img / 127.5) - 1.

In [40]:
def predict(model, image_name, threshold=0.2):
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = normalize(image)
    image = np.reshape(image,[1,input_shape[0], input_shape[1], n_channels])
    prediction = model.predict(image)[0]
    
    #print (prediction[prediction > threshold])
    #print (np.sum(prediction))
    
    
    indices = np.argwhere(prediction >= threshold).flatten()
    #print (indices)
    
    labels = " ".join(all_labels[indices].values)
    
    return labels

In [41]:
image_name = test_images_dir + test_images[0]
labels = predict(model, image_name)


print (labels)
#print (" ".join(labels.values))

/m/01g317 /m/07yv9 /m/09j2d


In [47]:
#sample_submission = pd.read_csv('./stage_1_sample_submission.csv')

submission = {'image_id':[], 'labels':[]}

for index, img in enumerate(test_images):
    #if index > 100:
    #    break
    
    image_name = test_images_dir + img
    
    if index % 1000 == 0:
        print ("processed {} images".format(index))
    
    image_id = img[:-4]
    
    labels = predict(model, image_name)
    
    submission['image_id'].append(image_id)
    submission['labels'].append(labels)
    
#sample_submission['image_id'] = submission['image_id']
#sample_submission['labels'] = submission['labels']
    
submission = pd.DataFrame(submission)

submission.head()

submission.to_csv('inception_450x450_1_epoch.csv', index=False)
    



processed 0 images
processed 1000 images


KeyboardInterrupt: 